In [9]:
import pandas as pd
import ujson
import pickle

In [10]:
fpath = '../data/SFPD.csv'
pddf = pd.read_csv(fpath)
pddf.head(1)
pddf.replace(to_replace=['FORGERY/COUNTERFEITING','DRUG/NARCOTIC','PORNOGRAPHY/OBSCENE MAT','LARCENY/THEFT'], value=['FORGERY-COUNTERFEITING','DRUG-NARCOTIC','PORNOGRAPHY-OBSCENE MAT','LARCENY-THEFT'], inplace=True, limit=None, regex=False, method='pad', axis=None)


In [11]:
hpdf = pd.read_csv('../data/SFlistingsN.csv')
hpdf.head()

,Unnamed: 0,averageListingPrice,medianListingPrice,numberOfProperties,type,weekEndingDate,neighborhoodName,TruliaNID,City,State
0,0,1120143,699000,7,All Properties,2009-07-04,Alamo Square,1386,San Francisco,CA
1,1,551000,535000,3,1 Bedroom Properties,2009-07-04,Alamo Square,1386,San Francisco,CA
2,2,553667,543000,3,1 Bedroom Properties,2009-07-04,Alamo Square,1386,San Francisco,CA
3,3,699000,699000,1,2 Bedroom Properties,2009-07-04,Alamo Square,1386,San Francisco,CA
4,4,799000,799000,1,3 Bedroom Properties,2009-07-04,Alamo Square,1386,San Francisco,CA


In [12]:
nbhoods = hpdf['neighborhoodName'].values
nbh = set(nbhoods)
print nbh

set(["Fisherman's Wharf", 'Ingleside Terrace', 'Financial District', 'Forest Knolls', 'West Portal', 'Golden Gate Heights', 'Cow Hollow', 'Bernal Heights', 'Buena Vista Park', 'Treasure Island', 'Duboce Triangle', 'Visitacion Valley', 'Marina', 'Merced Manor', 'Mount Davidson Manor', 'Saint Francis Wood', 'Forest Hill', 'Mission Bay', 'Chinatown', 'Outer Sunset', 'Stonestown', 'Outer Richmond', 'Cole Valley', 'Haight', 'South Beach', 'Portola', 'Sunnyside', 'Miraloma', 'Silver Terrace', 'Golden Gate Park', 'Lower Pacific Heights', 'Lake Street', 'Downtown', 'Lincoln Park - Lobos', 'Crocker Amazon', "Bernt's Knoll", 'Telegraph Hill', 'Inner Sunset', 'Outer Mission', 'Central Richmond', 'Twin Peaks', 'Mission Dolores', 'Inner Richmond', 'Japantown', 'Glen Park', 'Mission Terrace', 'Balboa Terrace', 'Hayes Valley', 'Merced Heights', 'Ingleside Heights', 'Nob Hill', 'Ashbury Heights', 'SoMa', 'Diamond Heights', 'Noe Valley', 'North Beach', 'Sherwood Forest', 'Bayview', 'Civic Center', 'Hun

In [13]:
ctype = pddf['Category'].values
ctp = set(ctype)
print ctp

set(['KIDNAPPING', 'WEAPON LAWS', 'SECONDARY CODES', 'WARRANTS', 'PROSTITUTION', 'EMBEZZLEMENT', 'SEX OFFENSES, NON FORCIBLE', 'SUICIDE', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'SUSPICIOUS OCC', 'FAMILY OFFENSES', 'DRUG-NARCOTIC', 'BAD CHECKS', 'DRUNKENNESS', 'GAMBLING', 'OTHER OFFENSES', 'LARCENY-THEFT', 'RECOVERED VEHICLE', 'FRAUD', 'ARSON', 'SEX OFFENSES, FORCIBLE', 'FORGERY-COUNTERFEITING', 'PORNOGRAPHY-OBSCENE MAT', 'TRESPASS', 'LOITERING', 'VANDALISM', 'NON-CRIMINAL', 'EXTORTION', 'LIQUOR LAWS', 'ASSAULT', 'TREA', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'BRIBERY', 'MISSING PERSON', 'DISORDERLY CONDUCT', 'RUNAWAY'])


In [14]:
## Clean Trulia dataset

#only use data for 2 bedroom poperties
hpdf = hpdf[hpdf['type'] == '2 Bedroom Properties']

#make east SF neighborhoods into SFPD districts, those are the ones that map decently/semi-easily with Trulia data. Get rid of the others
Bayview = ['Bayview', 'Central Waterfront', 'Hunters Point', 'Mission Bay', 'Portola', 'Potrero Hill', 'Silver Terrace']
Central = ['Central', 'Downtown', 'Financial District', 'Nob Hill', 'North Beach', 'Russian Hill', 'Telegraph Hill']
Mission = ['Mission', 'Mission Dolores', 'Noe Valley']
Northern = ['Northern', 'Alamo Square', 'Cow Hollow', 'Duboce Triangle', 'Hayes Valley', 'Lower Pacific Heights', 'Marina', 'Pacific Heights', 'Western Addition']
Southern = ['Southern', 'SoMa', 'South Beach']
Tenderloin = ['Tenderloin', 'Civic Center']

districts = [Bayview, Central, Mission, Northern, Southern, Tenderloin]

hpd = pd.DataFrame()

for district in districts:
    hpdnh = pd.DataFrame()
    
    for nh in district:
        hnh=hpdf[hpdf['neighborhoodName'] == nh]
        hnh['SFPD District'] = str(district[0]).upper()
        hpdnh=pd.concat([hpdnh,hnh])

    hpd=pd.concat([hpd,hpdnh])

#drop other columns
hpd.drop('Unnamed: 0', axis=1, inplace=True)
hpd.drop('medianListingPrice', axis=1, inplace=True)
hpd.drop('numberOfProperties', axis=1, inplace=True)
hpd.drop('TruliaNID', axis=1, inplace=True)

#average together multiple listings from the same week
hpd = hpd.set_index(pd.to_datetime(hpd['weekEndingDate']))
hpd = hpd.groupby(['weekEndingDate','SFPD District']).mean()
hpd = pd.DataFrame(hpd).reset_index()
hpd = hpd.set_index(pd.to_datetime(hpd['weekEndingDate']))
hpd.drop('weekEndingDate', axis=1, inplace=True)

/opt/conda/lib/python2.7/site-packages/IPython/kernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
rsfl = ['12M','6M', '3M','1M']

dist_list = ['TENDERLOIN']

for rsf in rsfl:
    for dist in dist_list:
        pdi = pddf[pddf['PdDistrict'] == dist.upper()]
        hpi = hpd[hpd['SFPD District'] == dist.upper()]

        pdct = pd.crosstab(pdi['Date'],pdi['Category'])
        pdct = pdct.set_index(pd.to_datetime(pdct.index))
        pdct['ALL'] = pdct.sum(axis=1)
        print pdct.head()
        break
    break



Category    ARSON  ASSAULT  BAD CHECKS  BRIBERY  BURGLARY  DISORDERLY CONDUCT  \
2003-01-01      0        7           0        0         0                   0   
2004-01-01      0        6           0        0         1                   0   
2005-01-01      0        9           0        0         1                   0   
2006-01-01      0       11           0        0         0                   0   
2007-01-01      0       10           0        0         0                   0   

Category    DRIVING UNDER THE INFLUENCE  DRUG-NARCOTIC  DRUNKENNESS  \
2003-01-01                            0              3            1   
2004-01-01                            0              5            0   
2005-01-01                            0              5            0   
2006-01-01                            0              3            2   
2007-01-01                            0              1            0   

Category    EMBEZZLEMENT  ...    SEX OFFENSES, NON FORCIBLE  STOLEN PROPERTY  \
2003-0

In [27]:
#Output Crime and Housing for each district
rsfl = ['12M','6M', '3M','1M']

dist_list = ['Bayview','Central','Mission','Northern','SOUTHERN','TENDERLOIN']

for rsf in rsfl:
    for dist in dist_list:
        pdi = pddf[pddf['PdDistrict'] == dist.upper()]
        hpi = hpd[hpd['SFPD District'] == dist.upper()]

        pdct = pd.crosstab(pdi['Date'],pdi['Category'])
        pdct = pdct.set_index(pd.to_datetime(pdct.index))
        pdct = pdct.resample(rsf, how='sum')
        pdct['ALL'] = pdct.sum(axis=1)
        pdct.drop(pdct.head(1).index, inplace=True)
        pdct.drop(pdct.tail(1).index, inplace=True)
        
        
        hpdc = hpi.resample(rsf, how='mean')

        #find average housing cost in all of SF
        hop = hpd.resample(rsf, how='mean')
        #append to hpdc
        hpdc['alp_allsf'] = hop['averageListingPrice']

        #merge dataframes for plotting
        merge = pdct.join(hpdc)
        merge = merge.drop(merge.head(1).index)
        merge = merge.drop(merge.tail(1).index)

        merge.columns.name = 'Date'

        fname = '../cast2/app/static/data/perd/{}-{}.csv'.format(dist.upper(), rsf.upper())
        merge.to_csv(fname)
        #print fname
        #print '%s %s Data Exported' % (dist.upper(), rsf.upper())
    

    pdi = pddf.copy()
    hpi = hpd.copy()
    dist = 'All'    
    pdct = pd.crosstab(pdi['Date'],pdi['Category'])
    pdct = pdct.set_index(pd.to_datetime(pdct.index))
    pdct = pdct.resample(rsf, how='sum')
    pdct['ALL'] = pdct.sum(axis=1)
    pdct.drop(pdct.head(1).index, inplace=True)
    pdct.drop(pdct.tail(1).index, inplace=True)
    
    hpdc = hpi.resample(rsf, how='mean')
    
    #find average housing cost in all of SF
    hop = hpd.resample(rsf, how='mean')
    #append to hpdc
    hpdc['alp_allsf'] = hop['averageListingPrice']

    #merge dataframes for plotting
    merge = pdct.join(hpdc)
    merge = merge.drop(merge.head(1).index)
    merge = merge.drop(merge.tail(1).index)

    merge.columns.name = 'Date'
    fname = '../cast2/app/static/data/perd/{}-{}.csv'.format(dist.upper(), rsf.upper())
    merge.to_csv(fname)
    #print '%s %s Data Exported' % (dist.upper(), rsf.upper())  
    print 'All %s Data Exported' % (rsf.upper()) 
print 'All Data Exported'

All 12M Data Exported
All 6M Data Exported
All 3M Data Exported
All 1M Data Exported
All Data Exported
